We will be using place names all over the place in the GeoKB. These will often be in addition to geographic coordinate claims on items, providing simple name-based characterization useful in all kinds of searches and reporting. We may have bots that operate internally to the GeoKB, building from coordinates to introduce derived place name claims. Or they may come in from source material.

Our GeoKB is likely not an authority for any place names, so we need to develop a series of bots to handle one-time and potentially periodic updating of place references. This notebook starts to build these out, and we may move to more purpose-built code for the bots in future.

# Properties and Classes

Working through places introduces new properties and classification items:

* geographic region (aka place)
    * Rather than try to be exhaustive on this, I stuck with a simple high level item (subclass of entity) for now. We can make this more complex in future if needed.
* ISO 3166-1 alpha-2 code
    * Specific property pulled from Wikidata as an identifier.

In [32]:
import os
import pandas as pd
import requests
from zipfile import ZipFile
from io import BytesIO
import swifter

from utils import (
    sparql_query,
    query_by_item_label
)

from wikibaseintegrator import WikibaseIntegrator, wbi_login, wbi_helpers
from wikibaseintegrator.wbi_config import config as wbi_config
from wikibaseintegrator.datatypes import Item, String, ExternalID, URL



In [34]:
wbi_config['MEDIAWIKI_API_URL'] = os.environ['MEDIAWIKI_API_URL']
wbi_config['SPARQL_ENDPOINT_URL'] = os.environ['SPARQL_ENDPOINT']
wbi_config['WIKIBASE_URL'] = os.environ['WIKIBASE_URL']

geokb_auth = wbi_login.Login(
    user=os.environ['WB_BOT_NAME'], 
    password=os.environ['WB_BOT_PASS']
)
wbi = WikibaseIntegrator(login=geokb_auth)

# Countries

For our mineral assessment use cases, we need items in GeoKB for every country where we have something like a mineral occurrence record or a document dealing with a mineral prospect in that country. So, we may as well input every country in the world from some reasonable source. There are lots of places we could go for this, but the Wikidata source of country records is pretty robust with a massive slate of properties for many countries because they are in such heavy use within the global knowledgebase.

Here, I run a query that pulls back just the essential bits we need right now:
* country name (ignoring aliases for now)
* country description
* two-character ISO3166 country code

We might need numeric codes at some point as well, but this is the minimum to start.

In [4]:
wd_country_query = """
SELECT ?country ?countryLabel ?countryDescription ?country_code WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?country wdt:P31 wd:Q6256.
  OPTIONAL {
  VALUES (?property) {
    (wdt:P297)
  }  
  ?country ?property ?country_code.
  }
}
"""

wd_countries = sparql_query(
    endpoint='https://query.wikidata.org/sparql',
    query=wd_country_query,
    output='dataframe'
)
wd_countries.head()

,country,countryLabel,countryDescription,country_code
0,http://www.wikidata.org/entity/Q258,South Africa,country in Southern Africa,ZA
1,http://www.wikidata.org/entity/Q262,Algeria,country in North Africa,DZ
2,http://www.wikidata.org/entity/Q265,Uzbekistan,sovereign state in Central Asia,UZ
3,http://www.wikidata.org/entity/Q298,Chile,country in South America,CL
4,http://www.wikidata.org/entity/Q334,Singapore,city-state in maritime Southeast Asia,SG


In [16]:
wd_countries[wd_countries.countryLabel.str.contains('United')]

,country,countryLabel,countryDescription,country_code
72,http://www.wikidata.org/entity/Q878,United Arab Emirates,sovereign state in Southwest Asia,AE
133,http://www.wikidata.org/entity/Q30,United States of America,country in North America,US
154,http://www.wikidata.org/entity/Q145,United Kingdom,country in north-west Europe,GB
186,http://www.wikidata.org/entity/Q1063498,United States of the Ionian Islands,Greek state,None


In [27]:
def get_label_id(label):
    label_query = query_by_item_label(label=label, include_aliases=False)
    geokb_results = wbi_helpers.execute_sparql_query(label_query)
    if len(geokb_results["results"]["bindings"]) == 1:
        return geokb_results["results"]["bindings"][0]["item"]["value"].split('/')[-1]

In [28]:
get_label_id('United States of America')

'Q161'

In [8]:
def add_item(label: str, description: str, aliases: list|str|None, claims: list|None):
    item = wbi.item.new()

    item.labels.set(language='en', value=label)
    item.descriptions.set(language='en', value=description)
    item.aliases.set(language='en', values=aliases)
    item.claims.add(claims)

    item.write()


In [11]:
instance_of = 'P1'
country_code_prop = 'P10'
country_class = 'Q27'

instance_of_country_claim = Item(
    prop_nr=instance_of, 
    value=country_class
)

for index, row in wd_countries[wd_countries.country_code != 'ZA'].iterrows():
    print("PROCESSING:", row.countryLabel)
    claims = [instance_of_country_claim]
    if row.country_code:
        country_code_claim = String(
            prop_nr=country_code_prop,
            value=row.country_code
        )
        claims.append(country_code_claim)

    add_item(
        label=row.countryLabel,
        description=row.countryDescription,
        aliases=row.country_code,
        claims=claims
    )


PROCESSING: Algeria
None
PROCESSING: Uzbekistan
None
PROCESSING: Chile
None
PROCESSING: Singapore
None
PROCESSING: Liechtenstein
None
PROCESSING: Bahrain
None
PROCESSING: Armenia
None
PROCESSING: Australia
None
PROCESSING: Argentina
None
PROCESSING: Peru
None
PROCESSING: North Korea
None
PROCESSING: Cambodia
None
PROCESSING: East Timor
None
PROCESSING: Chad
None
PROCESSING: New Zealand
None
PROCESSING: India
None
PROCESSING: Tuvalu
None
PROCESSING: Samoa
None
PROCESSING: Vanuatu
None
PROCESSING: Solomon Islands
None
PROCESSING: Papua New Guinea
None
PROCESSING: Palau
None
PROCESSING: Nauru
None
PROCESSING: Federated States of Micronesia
None
PROCESSING: Kiribati
None
PROCESSING: Mongolia
None
PROCESSING: Marshall Islands
None
PROCESSING: Fiji
None
PROCESSING: Venezuela
None
PROCESSING: Guyana
None
PROCESSING: Paraguay
None
PROCESSING: Colombia
None
PROCESSING: Ecuador
None
PROCESSING: Bolivia


Service unavailable (HTTP Code 502). Sleeping for 60 seconds.


None
PROCESSING: Trinidad and Tobago
None
PROCESSING: Saint Vincent and the Grenadines
None
PROCESSING: Saint Kitts and Nevis
None
PROCESSING: Saint Lucia
None
PROCESSING: Jamaica
None
PROCESSING: Grenada
None
PROCESSING: Guatemala
None
PROCESSING: The Bahamas
None
PROCESSING: Honduras
None
PROCESSING: Antigua and Barbuda
None
PROCESSING: Dominican Republic
None
PROCESSING: Dominica
None
PROCESSING: Haiti
None
PROCESSING: Iran
None
PROCESSING: El Salvador
None
PROCESSING: Iraq
None
PROCESSING: Costa Rica
None
PROCESSING: Israel
None
PROCESSING: Yemen
None
PROCESSING: Jordan
None
PROCESSING: Nicaragua
None
PROCESSING: Kyrgyzstan
None
PROCESSING: Laos
None
PROCESSING: Lebanon
None
PROCESSING: Maldives
None
PROCESSING: Myanmar
None
PROCESSING: Nepal
None
PROCESSING: Oman
None
PROCESSING: Pakistan
None
PROCESSING: Qatar
None
PROCESSING: Saudi Arabia
None
PROCESSING: Sri Lanka
None
PROCESSING: Syria
None
PROCESSING: Tajikistan
None
PROCESSING: Taiwan
None
PROCESSING: Thailand
None
PROCESSIN

# Sub-country Units

We at least also need U.S. States and Territories and may need states/provinces from other countries as I know those show up in some of our data. We'll also probably want U.S. Counties. We'll introduce additional sub-country administrative units as they come up in our data sources, working through the same overall logic on identifying and processing reasonable reference sources.

In [29]:
wd_us_states_query = """
SELECT DISTINCT ?state ?stateLabel ?stateDescription ?iso3166 ?fips_alpha ?fips_numeric WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
  ?state wdt:P31 wd:Q35657.
  OPTIONAL {
  VALUES (?propertyISO3166) {
    (wdt:P300)
  }  
  ?state ?propertyISO3166 ?iso3166.
  VALUES (?propertyFipsNumeric) {
    (wdt:P5087)
  }  
  ?state ?propertyFipsNumeric ?fips_numeric.
  VALUES (?propertyFipsAlpha) {
    (wdt:P5086)
  }  
  ?state ?propertyFipsAlpha ?fips_alpha.
  }
}

"""

wd_us_states = sparql_query(
    endpoint='https://query.wikidata.org/sparql',
    query=wd_us_states_query,
    output='dataframe'
)
wd_us_states.head()

,state,stateLabel,stateDescription,iso3166,fips_alpha,fips_numeric
0,http://www.wikidata.org/entity/Q1166,Michigan,state of the United States of America,US-MI,MI,26
1,http://www.wikidata.org/entity/Q1588,Louisiana,state of the United States of America,US-LA,LA,22
2,http://www.wikidata.org/entity/Q1649,Oklahoma,state of the United States of America,US-OK,OK,40
3,http://www.wikidata.org/entity/Q99,California,state of the United States of America,US-CA,CA,06
4,http://www.wikidata.org/entity/Q1428,Georgia,state of the United States of America,US-GA,GA,13


In [35]:
reference_url = 'P4'
us_state = 'Q229'
is_located_in = 'P11'
us = 'Q161'
iso_3166_2 = 'P12'
fips_alpha = 'P13'
fips_numeric = 'P14'

state_instance_of_claim = Item(
    prop_nr=instance_of,
    value=us_state
)

located_in_us_claim = Item(
    prop_nr=is_located_in,
    value=us
)

for index, row in wd_us_states.iterrows():
    references = [URL(
        prop_nr=reference_url,
        value=row.state
    )]

    iso_3166_claim = ExternalID(
        prop_nr=iso_3166_2,
        value=row.iso3166,
        references=references
    )

    fips_alpha_claim = ExternalID(
        prop_nr=fips_alpha,
        value=row.fips_alpha,
        references=references
    )

    fips_numeric_claim = ExternalID(
        prop_nr=fips_numeric,
        value=row.fips_numeric,
        references=references
    )

    claims = [
        state_instance_of_claim,
        located_in_us_claim,
        iso_3166_claim,
        fips_alpha_claim,
        fips_numeric_claim
    ]

    add_item(
        label=row.stateLabel,
        description=row.stateDescription,
        aliases=row.fips_alpha,
        claims=claims
    )


# GNIS Places

There are likely some key feature classes from the GNIS that we should establish a relationship with. While the GNIS is *A* source for some features, it may not be the *BEST* source for our purposes. For instance, should we simply pull all GNIS named mines as things we are likely to encounter and need to link to at some point, should we pull mines from USMIN as a more contemporary source, should we pull USMIN mines and then get other mines from GNIS, when and how should we handle mine references in MRDS that exist as mineral occurrences...lots of questions to work through. One way or the other, the whole point of the GeoKB effort is to help resolve these issues, giving us a living source with a set of rules for how things come into the knowledgebase and work out their differences.

The codeblocks below pull the GNIS national file and read it into a dataframe. We look at the feature classes present and pull out mines as an example.

In [65]:
gnis_national_file = 'https://geonames.usgs.gov/docs/stategaz/NationalFile.zip'
r_gnis_national_file = requests.get(gnis_national_file)
z = ZipFile(BytesIO(r_gnis_national_file.content))
# We know it's the first/only file and it's delimited with pipe
gnis_national = pd.read_csv(z.open(z.namelist()[0]), sep='|')

In [66]:
gnis_national.FEATURE_CLASS.unique()

array(['Stream', 'Valley', 'Park', 'Crossing', 'Cape', 'Summit', 'School',
       'Canal', 'Mine', 'Plain', 'Locale', 'Spring', 'Basin', 'Reservoir',
       'Well', 'Populated Place', 'Flat', 'Cliff', 'Lake', 'Arch', 'Gap',
       'Range', 'Dam', 'Bench', 'Ridge', 'Trail', 'Cemetery', 'Hospital',
       'Building', 'Arroyo', 'Falls', 'Bay', 'Swamp', 'Area', 'Pillar',
       'Rapids', 'Island', 'Bend', 'Church', 'Bar', 'Bridge', 'Civil',
       'Channel', 'Tunnel', 'Military', 'Tower', 'Levee', 'Crater',
       'Lava', 'Harbor', 'Woods', 'Airport', 'Reserve', 'Slope', 'Gut',
       'Beach', 'Post Office', 'Forest', 'Oilfield', 'Census', 'Glacier',
       'Sea', 'Isthmus', 'Unknown'], dtype=object)

In [67]:
mine_features = gnis_national[gnis_national.FEATURE_CLASS == 'Mine']
mine_features.head(10)

,FEATURE_ID,FEATURE_NAME,FEATURE_CLASS,STATE_ALPHA,STATE_NUMERIC,COUNTY_NAME,COUNTY_NUMERIC,PRIMARY_LAT_DMS,PRIM_LONG_DMS,PRIM_LAT_DEC,PRIM_LONG_DEC,SOURCE_LAT_DMS,SOURCE_LONG_DMS,SOURCE_LAT_DEC,SOURCE_LONG_DEC,ELEV_IN_M,ELEV_IN_FT,MAP_NAME,DATE_CREATED,DATE_EDITED
44,444,Yucca Mine,Mine,AZ,4,Mohave,15.0,343909N,1142231W,34.652509,-114.375235,NaN,NaN,NaN,NaN,451.0,1480.0,Topock,02/08/1980,05/01/2006
70,470,Abe Lincoln Mine,Mine,AZ,4,Yavapai,25.0,340244N,1123232W,34.045586,-112.542118,NaN,NaN,NaN,NaN,1192.0,3911.0,Morgan Butte,02/08/1980,NaN
73,473,Abril Mine,Mine,AZ,4,Cochise,3.0,315429N,1095929W,31.908140,-109.991459,NaN,NaN,NaN,NaN,2031.0,6663.0,Cochise Stronghold,02/08/1980,NaN
83,483,Adams Mine,Mine,AZ,4,Mohave,15.0,345808N,1142335W,34.968892,-114.393014,NaN,NaN,NaN,NaN,659.0,2162.0,Boundary Cone,02/08/1980,NaN
118,519,Aguinaldo Mine,Mine,AZ,4,Pima,19.0,315508N,1111712W,31.918971,-111.286767,NaN,NaN,NaN,NaN,1139.0,3737.0,Stevens Mountain,02/08/1980,NaN
146,547,Alabama Mine,Mine,AZ,4,Mohave,15.0,352027N,1133603W,35.340831,-113.600772,NaN,NaN,NaN,NaN,1501.0,4924.0,Valentine SE,02/08/1980,NaN
162,563,Alaska Mine,Mine,AZ,4,Maricopa,13.0,334342N,1131854W,33.728366,-113.314918,NaN,NaN,NaN,NaN,572.0,1877.0,Weldon Hill,02/08/1980,NaN
163,564,Ajax Mine,Mine,AZ,4,Cochise,3.0,320048N,1091243W,32.013422,-109.212006,NaN,NaN,NaN,NaN,1473.0,4833.0,Blue Mountain,02/08/1980,NaN
174,575,Alcyone Mine,Mine,AZ,4,Mohave,15.0,345934N,1142425W,34.992781,-114.406904,NaN,NaN,NaN,NaN,664.0,2178.0,Boundary Cone,02/08/1980,NaN
187,588,Alice Mine,Mine,AZ,4,Pinal,21.0,330756N,1105525W,33.132284,-110.923722,NaN,NaN,NaN,NaN,935.0,3068.0,Hot Tamale Peak,02/08/1980,NaN


# Adding to GeoKB

With this information in hand, we can stub out basic country reference items with the bare minimum of information needed to establish linkages. We can also include the Wikidata identifier for future use in retrieving additional information when it comes up in a use case.

## Stuck Again!

I'm stuck here again because of whatever is going on with the API. I can add a single claim to an item, but as soon as I do that, the addClaim() function fails on the next claim I try to add with a different error. Essentially, once claims are on an item, that item is alive and well in Wikibase (UI and SPARQL) but is dead to the API. Argh!

In [3]:
geokb_site = get_wb('geokb')

In [ ]:
for index, row in wd_countries.head().iterrows():
    claims = [
        {
            'property_id': 'P2',
            'object': 'Q37',
            'prov_statement': 'Adding classification for country'
        },
        {
            'property_id': 'P26',
            'object': row.country_code,
            'prov_statement': 'Adding country code'
        }
    ]

    display(claims)

    # process_item(
    #     site=geokb_site,
    #     label=row.countryLabel,
    #     description=row.countryDescription,
    #     claims=claims,
    #     prov_statement='Adding country item borrowed from Wikidata'
    # )